# Modelli di Equazioni Strutturali Multilivello {#sec-sem-multilevel}

In [1]:
source(here::here("code", "_common.R"))

suppressPackageStartupMessages({
    library("lavaan")
    library("semPlot")
    library("knitr")
    library("markdown")
    library("patchwork")
    library("lme4")
    library("here")
})

I dati raccolti nella ricerca psicologica e nelle scienze sociali spesso presentano complesse strutture multi-livello, dove le informazioni sono organizzate in gruppi o cluster, e le osservazioni all'interno di questi cluster tendono a essere correlate tra loro. Questo aspetto è fondamentale poiché, in molte situazioni, i modelli classici di equazioni strutturali non sono adeguati per l'analisi di tali dati. Il motivo principale è che questi modelli tradizionali non tengono conto delle correlazioni intrinseche presenti nei dati organizzati su più livelli. L'uso dei modelli SEM tradizionali, senza adeguare la loro struttura ai dati multi-livello, può portare a stime che non sono né valide né efficienti.

Nel contesto della modellazione delle equazioni strutturali multilivello (multilevel SEM), si adotta un approccio che prevede l'utilizzo di variabili latenti per distinguere la variazione tra i cluster e quella all'interno dei cluster stessi. In questa prospettiva, le variabili osservate vengono considerate come influenzate congiuntamente da fattori latenti che incidono sui dati all'interno di ogni cluster e da fattori latenti che influenzano le differenze tra i diversi cluster. 

In sostanza, ciò significa che il modello include variabili latenti progettate per catturare le differenze di gruppo (tra i cluster) e le variazioni individuali (all'interno di ciascun cluster), consentendo un'analisi più precisa e rappresentativa dei dati complessi tipici delle scienze sociali e psicologiche. Mediante questa strategia, siamo in grado di comprendere in modo più approfondito sia la variazione sistematica tra i gruppi che quella individuale all'interno dei gruppi stessi, offrendo così un quadro più completo e informativo delle dinamiche sottostanti ai fenomeni studiati.

In questo capitolo introdurremo brevemente l'implementazione in `lavaan` per l'analisi SEM multilivello. Utilizzeremo un set di dati artificiali ricavato dal sito di MPlus.

In [2]:
dat <- read.table("http://statmodel.com/usersguide/chap9/ex9.6.dat")
names(dat) <- c("y1", "y2", "y3", "y4", "x1", "x2", "w", "clus")
head(dat)

,y1,y2,y3,y4,x1,x2,w,clus
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,2.203250,1.858861,1.738477,2.244863,1.142800,-0.796987,-0.149501,1
2,1.934917,2.127876,0.083120,2.509436,1.949033,-0.122764,-0.149501,1
3,0.321955,0.977231,-0.835405,0.558367,-0.716481,-0.767064,-0.149501,1
4,0.073154,-1.743092,-2.310271,-1.514332,-2.649131,0.637570,-0.149501,1
5,-1.214906,0.452618,0.372610,-1.790372,-0.262916,0.302564,-0.149501,1
6,0.298330,-1.820272,0.561335,-2.090582,-0.944963,1.363045,0.319335,2


Il data frame è costituito da 1000 righe:

In [9]:
dim(dat)

[1] 1000    8

Ci sono soltanto 110 cluster (`clus`), il che significa che ci sono misure ripetute per ciascun cluster (possiamo immaginare che i cluster corrispondano ai soggetti):

In [10]:
length(unique(dat$clus))

[1] 110

Analizzeremo questi dati mediante un modello di equazioni strutturali multilivello (SEM). Iniziamo a definire il modello SEM appropriato per questi dati.

In [11]:
model <- "
    level: 1
        fw =~ y1 + y2 + y3 + y4
        fw ~ x1 + x2

    level: 2
        fb =~ y1 + y2 + y3 + y4

    # optional
    y1 ~~ 0*y1
    y2 ~~ 0*y2
    y3 ~~ 0*y3
    y4 ~~ 0*y4
    fb ~ w
"

1. **Livello 1:** Il primo livello rappresenta le misurazioni individuali. Nel modello, `fw` è un fattore latente che viene definito dalle quattro variabili osservate `y1`, `y2`, `y3`, `y4`. Queste variabili sono influenzate da due predittori `x1` e `x2`. Questo livello cattura la variazione a livello individuale.

2. **Livello 2:** Il secondo livello considera le differenze tra i gruppi o i cluster (indicati dalla variabile `clus`). Qui, `fb` è un altro fattore latente definito dalle stesse variabili osservate `y1`, `y2`, `y3`, `y4`. La variabile `w` agisce come un predittore a livello di cluster.

### Coefficiente di Correlazione Intraclass 

Lo scopo dell'analisi SEM Multilivello è quello di calcolare il Coefficiente di Correlazione Intraclasse (ICC). L'ICC è una misura statistica usata in studi dove i dati sono strutturati in gruppi o cluster. L'ICC valuta la somiglianza o l'omogeneità delle misurazioni all'interno dei gruppi. In termini più tecnici, l'ICC quantifica la proporzione di varianza totale nei dati che può essere attribuita alle differenze tra i gruppi. Più alto è l'ICC, maggiore è l'influenza del raggruppamento sulle misurazioni, indicando che una parte sostanziale della varianza osservata nei dati deriva dalle differenze tra i gruppi piuttosto che dalle variazioni individuali all'interno di essi.

#### ICC nei Modelli SEM Multilivello

Calcolare l'ICC (*Intra-Class Correlation*) per ciascuna variabile osservata in un modello SEM multilivello è cruciale per comprendere la struttura dei dati. Un ICC elevato per una variabile specifica indica che la posizione di un individuo all'interno di un determinato gruppo o cluster ha un notevole impatto su quella misurazione. Questo suggerisce che le differenze tra i gruppi (come individui, scuole o unità familiari) sono aspetti cruciali che devono essere considerati nell'analisi dei dati.

In pratica, l'ICC aiuta a determinare se è opportuno adottare un approccio di analisi multilivello. Un ICC basso suggerisce che potrebbe non essere necessario utilizzare modelli complessi che considerano la struttura a cluster dei dati. Al contrario, un ICC alto indica che è fondamentale modellare accuratamente la struttura multilivello dei dati per evitare stime distorte e conclusioni erronee. Utilizzando l'ICC come guida, gli analisti possono decidere se e in che misura integrare l'approccio multilivello per ottenere una comprensione più accurata e affidabile dei fenomeni studiati.

#### Calcolo dell'ICC con `lmer`

Per iniziare, poniamoci il problema di calcolare ICC mediante un modello lineare multilivello. Svolgeremo questi calcoli con `lmer`. Il modello `lmer` considera ogni variabile osservata separatamente, fornendo un'analisi indipendente per ciascuna. Per `y1`, per esempio, abbiamo:

In [12]:
model_lmer <- lmer(y1 ~ 1 + (1 | clus), data = dat)

Calcoliamo l'ICC:

In [13]:
 varianza_cluster <- VarCorr(model_lmer)$clus[1]
 varianza_residua <- attr(VarCorr(model_lmer), "sc")^2
 ICC <- varianza_cluster / (varianza_cluster + varianza_residua)
 ICC

[1] 0.1295362

Il Coefficiente di Correlazione Intraclass (ICC) di 0.129 per la variabile `y1` significa che circa il 12.9% della variazione totale in `y1` è ascrivibile alle differenze tra gli studenti, considerati come unità separate o cluster individuali. Questa percentuale relativamente modesta della variazione totale suggerisce che le caratteristiche o i comportamenti individuali degli studenti spiegano solo una piccola parte della variazione osservata in `y1`. 

Un ICC di questo livello, che si può considerare relativamente basso, implica che la maggior parte della variazione nella variabile non è legata in modo sostanziale alle differenze tra gli studenti. Questo può essere indicativo del fatto che altri fattori, esterni alle caratteristiche individuali degli studenti, giocano un ruolo più determinante. In un contesto educativo, ad esempio, questo potrebbe suggerire che elementi come l'ambiente scolastico, le metodologie didattiche impiegate, o le specificità del programma di studi, hanno un impatto maggiore sulla variazione di `y1` rispetto alle differenze individuali tra gli studenti. 

Calcoliamo ora l'ICC con un modello SEM multilivello.

In [14]:
 fit <- sem(model,
     data = dat,
     cluster = "clus",
     fixed.x = FALSE
 )

In [15]:
summary(fit) |>
    print()

lavaan 0.6.17 ended normally after 27 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                        26

  Number of observations                          1000
  Number of clusters [clus]                        110

Model Test User Model:
                                                      
  Test statistic                                 3.863
  Degrees of freedom                                17
  P-value (Chi-square)                           1.000

Parameter Estimates:

  Standard errors                             Standard
  Information                                 Observed
  Observed information based on                Hessian


Level 1 [within]:

Latent Variables:
                   Estimate  Std.Err  z-value  P(>|z|)
  fw =~                                               
    y1                1.000                           
    y2                0.999    0.033  

In [16]:
fitMeasures(fit) |>
    print()

                 npar                  fmin                 chisq 
               26.000                 3.913                 3.863 
                   df                pvalue        baseline.chisq 
               17.000                 1.000              3283.563 
          baseline.df       baseline.pvalue                   cfi 
               24.000                 0.000                 1.000 
                  tli                  nnfi                   rfi 
                1.006                 1.006                 0.998 
                  nfi                  pnfi                   ifi 
                0.999                 0.707                 1.004 
                  rni                  logl     unrestricted.logl 
                1.004             -9527.429             -9525.497 
                  aic                   bic                ntotal 
            19106.857             19234.459              1000.000 
                 bic2                 rmsea        rmsea.ci.lo

Quando calcoliamo l'ICC con `lavaan`, ottieniamo valori separati per ciascuna variabile osservata nel contesto del modello multilivello. Questi valori riflettono quanto la varianza in ogni variabile sia attribuibile alle differenze tra i gruppi, ma all'interno del framework più ampio delle relazioni specificate nel modello SEM. L'ICC per ciascuno dei 4 item si trova nel modo seguente:

In [17]:
lavInspect(fit, "icc") |>
    print()

   y1    y2    y3    y4    x1    x2 
0.125 0.121 0.106 0.115 0.000 0.000 


Nel caso di `y1`, la stima di ICC fornita dal modello SEM multilivello è molto simile al risultato ottenuto con `lmer`.

## Considerazioni Conclusive

In questo capitolo, abbiamo illustrato il modello di equazioni strutturali multilivello utilizzando `lavaan`. Come evidenziato dall'esempio, l'implementazione in `lavaan` è molto diretta, richiedendo solo l'inclusione dell'opzione `cluster` nella funzione `sem`. È importante sottolineare che, al momento, `lavaan` supporta solo modelli SEM a due livelli. 

Nell'ambito dei modelli SEM multilivello, abbiamo visto come l'interpretazione dei coefficienti di correlazione intra-classe (ICC) possa fornire intuizioni significative sulla variazione dei dati all'interno di gruppi o cluster. Un ICC basso, come quello osservato nell'esempio (0.129), indica che una porzione minore della variazione totale è attribuibile alle differenze tra i cluster. Nel contesto specifico dei nostri dati, dove ogni studente è considerato un cluster individuale, ciò suggerisce che fattori esterni agli studenti stessi potrebbero giocare un ruolo più significativo nella variazione osservata rispetto alle caratteristiche individuali degli studenti.

In conclusione, la modellazione di equazioni strutturali multilivello è uno strumento potente e flessibile nell'analisi di dati strutturati gerarchicamente. `lavaan`, sebbene limitato ai modelli a due livelli, fornisce un approccio accessibile e diretto per questi tipi di analisi. Per modelli più complessi e a più livelli, Mplus e Stata offrono soluzioni alternative che possono gestire una gamma più ampia di esigenze analitiche. 